<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Roselynn/Copy_of_Copy_of_592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
# This mouunts your google drive to the current runtime
drive.mount('/content/mnt')
# We define a notebook path
nb_path = '/content/notebooks'
# We create a symbolic link from our drive's default "Colab Notebooks" folder to nb_path
os.symlink('/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid', nb_path)
# Insert nb path
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [2]:
!unzip '/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/RoverTrainingData_Grid.zip'

Archive:  /content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/RoverTrainingData_Grid.zip
   creating: RoverTrainingData_Grid/IMG/
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_540.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_597.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_671.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_732.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_801.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_867.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_935.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_001.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_067.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_135.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_201.jpg  
  i

In [3]:
print(len('RoverTraningData_Grid/IMG'))

25


In [10]:
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
from torch import int64
#img_dir = "Images"
#img_dir = "/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/IMG"
img_dir = "RoverTrainingData_Grid/IMG"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [11]:
print(len(Images))

2297


In [13]:
controlData = np.loadtxt('RoverTrainingData_Grid/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [14]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
steerDataTrain = torch.tensor(controlData[0:799, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
steerDataTest = torch.tensor(controlData[800:999, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

In [15]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=128, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=128, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=128, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=128, shuffle=False)

In [16]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
        self.activation = torch.nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        #transformed_x4 = self.activation(transformed_x4)
        return transformed_x4

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [17]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(20):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 27533.692977730716 and test loss 15988.39404296875
Epoch 1 finished with train loss 6966.214694976807 and test loss 4233.5694580078125
Epoch 2 finished with train loss 3094.0784029279434 and test loss 2661.314208984375
Epoch 3 finished with train loss 1006.369499206543 and test loss 927.3950805664062
Epoch 4 finished with train loss 329.26494096006667 and test loss 452.833251953125
Epoch 5 finished with train loss 176.23236785616194 and test loss 90.68535614013672
Epoch 6 finished with train loss 101.88367380414691 and test loss 4.139460802078247
Epoch 7 finished with train loss 27.24554567677634 and test loss 37.84836769104004
Epoch 8 finished with train loss 12.95356948886599 and test loss 13.327728748321533
Epoch 9 finished with train loss 11.055507251194545 and test loss 0.23243080079555511
Epoch 10 finished with train loss 3.4691412236009325 and test loss 3.8784501552581787
Epoch 11 finished with train loss 1.5350472969668252 and test loss 1.911214

In [18]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 32919.38620550292 and test loss 24582.5537109375
Epoch 1 finished with train loss 10287.675894601005 and test loss 320.75048828125
Epoch 2 finished with train loss 3438.958012172154 and test loss 162.96888732910156
Epoch 3 finished with train loss 1322.7038051060267 and test loss 102.69634246826172
Epoch 4 finished with train loss 344.3936917270933 and test loss 512.948974609375
Epoch 5 finished with train loss 183.48066035338812 and test loss 50.383432388305664
Epoch 6 finished with train loss 124.44194684709821 and test loss 88.04115676879883
Epoch 7 finished with train loss 27.660146372658865 and test loss 18.027864456176758
Epoch 8 finished with train loss 31.206339154924667 and test loss 16.907869815826416
Epoch 9 finished with train loss 5.42016127705574 and test loss 6.466007947921753
Epoch 10 finished with train loss 8.358921527862549 and test loss 3.7481483221054077
Epoch 11 finished with train loss 1.404073919568743 and test loss 2.31604373455

In [21]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
#img_dir = "Images"
img_dir = "RoverTrainingData_Grid/IMG"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)

for i in np.linspace(1, 999, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle))

correct steer: 0.0 predicted steer: tensor([[-0.2374]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.3650]], grad_fn=<AddmmBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.0189]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.0810]], grad_fn=<AddmmBackward0>)
correct steer: -0.7830426666666667 predicted steer: tensor([[-0.1033]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.3209]], grad_fn=<AddmmBackward0>)
correct steer: -1.0 predicted steer: tensor([[-0.0676]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.5229]], grad_fn=<AddmmBackward0>)
correct steer: 0.6324920666666666 predicted steer: tensor([[-0.1207]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predicted throttle: tensor([[0.4707]], grad_fn=<AddmmBackward0>)
correct steer: 0.35304953333333333 predicted steer: tensor([[0.0971]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predi